In [ ]:
import pandas as pd

# Read the CSV of Value of Energy Cost Savings and check the head of the dataframe
df = pd.read_csv("C:/Users/leab3/Documents/Data Incubator/Pre-Project/Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_-_FY2020.csv", parse_dates=True)
df.head()

In [ ]:
# 1. How many companies are listed in the file?
len(pd.unique(df["Company Name"]))

In [ ]:
# 2. What is the total number of jobs created for businesses in Queens?
df[df['Borough']=='Queens']['Job created'].sum()

In [ ]:
# 3. How many different unique email domains names are there in the data set?
len(pd.unique(df['company email'].str.split('@').str[-1]))

In [ ]:
# 4. Considering only NTAs with at least 5 listed businesses, what is the average total savings and the total jobs created for each NTA?
NTA_over_5 = df[df['NTA'].map(df['NTA'].value_counts())>5].groupby(['NTA'],as_index=False).agg({'Total Savings': ['mean'], 'Job created': ['count']})
NTA_over_5.columns = NTA_over_5.columns.get_level_values(0)
NTA_over_5.columns = ['NTA','Avg Total Savings','Jobs created']

In [ ]:
# 5. Save result as CSV
NTA_over_5.to_csv("C:/Users/leab3/Documents/Data Incubator/Pre-Project/Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_-_FY2020_NAT_over_5.csv")

In [ ]:
# Day 5: Plotting in MatPlotLib
from calendar import month
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns

# Set up standard scale scatter plot and show
plt.scatter(NTA_over_5['Avg Total Savings'],NTA_over_5['Jobs created'])
plt.xlabel('Avg Total Savings')
plt.ylabel('Jobs Created')
plt.title('Scatter Plot - Standard Scale')
plt.show()

# Set up logarithmic scale scatter plot and show
print("Scatter Plot - Standard Scale: ")
NTA_over_5['Log Avg Total Savings']=np.log(NTA_over_5['Avg Total Savings'])
plt.scatter(NTA_over_5['Log Avg Total Savings'],NTA_over_5['Jobs created'])
plt.xlabel('Total Savings')
plt.ylabel('Jobs Created')
plt.title('Scatter Plot - Logarithmic Scale')
plt.show()

# Set up histogram of the log of total average savings
plt.hist(NTA_over_5['Log Avg Total Savings'])
plt.title('Histogram')
plt.show()

# Create month and year column
df['Effective Date']=pd.to_datetime(df['Effective Date'])
df['month'] = pd.DatetimeIndex(df['Effective Date']).month
df['year'] = pd.DatetimeIndex(df['Effective Date']).year

# Aggregate count of jobs created by month
df_month_agg = df.groupby(['month','year'],as_index=False).agg({'Job created': ['count']}).sort_values(by=['year','month'])
df_month_agg.columns = df_month_agg.columns.get_level_values(0)
df_date_agg = df.groupby(['Effective Date'],as_index=False).agg({'Job created': ['count']}).sort_values(by=['Effective Date'])
df_date_agg.columns = df_date_agg.columns.get_level_values(0)

# Line plot
plt.plot(df_month_agg['month'],df_month_agg['Job created'])
plt.title('Line plot of count of jobs created by month')
plt.xlabel('Month')
plt.ylabel('Jobs Created')
plt.show()

# Month/Year plot
sns.relplot(data=df_month_agg, x="month", y="Job created", hue="year", kind="line")
plt.xlabel('Month')
plt.ylabel('Jobs Created')
plt.title('Line plot - Total jobs created by month')
plt.show()